In [1]:
import numpy as np
from scipy.ndimage import median_filter
from sklearn.linear_model import RANSACRegressor
from filterpy.kalman import KalmanFilter

In [3]:
def remove_outliers(data, threshold=3):
    mean = np.mean(data)
    std = np.std(data)
    filtered_data = [x for x in data if (mean -threshold * std <x< mean + threshold * std)]
    return filtered_data

In [4]:
def apply_median_filter(data, filter_size=3):
    filtered_data = median_filter(data, size=filter_size)
    return filtered_data

In [9]:
# Smoothing using Moving Average
def apply_moving_average(data, window_size=3):
    cumsum = np.cumsum(np.insert(data, 0, 0))
    smoothed_data = (cumsum[window_size:] - cumsum[:-window_size]) / float(window_size)
    return smoothed_data

In [10]:
# Robust Regression
def perform_robust_regression(x, y):
    ransac = RANSACRegressor()
    ransac.fit(x.reshape(-1, 1), y.reshape(-1, 1))
    inlier_mask = ransac.inlier_mask_
    outlier_mask = np.logical_not(inlier_mask)
    inlier_x = x[inlier_mask]
    inlier_y = y[inlier_mask]
    outlier_x = x[outlier_mask]
    outlier_y= y[outlier_mask]
    return inlier_x, inlier_y, outlier_x, outlier_y

In [11]:
def apply_kalman_filter(data, measurement_noise, process_noise):
    kf = KalmanFilter(dim_x=2, dim_z=1)
    kf.x = np.array([data[0], 0]) # Initial state: (position, velocity]
    kf.P *= 1000
    kf.R = measurement_noise # Measurement noise covariance
    kf.Q = process_noise # Process noise covariance
    kf.F = np.array([[1, 1], [0, 1]]) # State transition matrix
    kf.H = np.array([[1, 0]]) # Measurement matrix
    filtered_data = []
    for measurement in data:
        kf.predict()
        kf.update(np.array([measurement]))
        filtered_data.append(kf.x[0])

    return filtered_data

In [12]:
data = [1, 2, 3, 100, 4, 5, 200, 6, 7, 8]
# Outlier Removal
outlier_removed_data = remove_outliers(data)
print("Outlier Removed Data:", outlier_removed_data)
# Median Filtering
median_filtered_data = apply_median_filter(data)
print("Median Filtered Data:", median_filtered_data)
# Smoothing using Moving Average
moving_average_data = apply_moving_average(data)
print("Moving Average Data:", moving_average_data)
# Robust Regression
x= np.array([1, 2, 3, 4, 5, 6, 7, 8])
y= np.array([10, 20, 30, 400, 50, 60, 700, 80])
inlier_x, inlier_y, outlier_x, outlier_y= perform_robust_regression(x, y)
print("Inliers:")
for i in range(len(inlier_x)):
    print(f"x: {inlier_x[i]}, y: {inlier_y[i]}")
print("Outliers:")
for i in range(len(outlier_x)):
    print(f"x: {outlier_x[i]}, y: {outlier_y[i]}")
# Kalman Filtering
data = [1, 2, 3, 100, 4, 5, 200, 6, 7, 8]
measurement_noise = 1.0
process_noise = 0.1
kalman_filtered_data = apply_kalman_filter(data, measurement_noise, process_noise)
print("Original Data:", data)
print("Kalman Filtered Data:", kalman_filtered_data)

Outlier Removed Data: [1, 2, 3, 100, 4, 5, 200, 6, 7, 8]
Median Filtered Data: [1 2 3 4 5 5 6 7 7 8]
Moving Average Data: [ 2.         35.         35.66666667 36.33333333 69.66666667 70.33333333
 71.          7.        ]
Inliers:
x: 1, y: 10
x: 2, y: 20
x: 3, y: 30
x: 5, y: 50
x: 6, y: 60
x: 8, y: 80
Outliers:
x: 4, y: 400
x: 7, y: 700
Original Data: [1, 2, 3, 100, 4, 5, 200, 6, 7, 8]
Kalman Filtered Data: [1.0, 1.9980134053394396, 2.998855729700037, 72.36135112417385, 41.010240338180175, 22.810255784942303, 121.72816285454566, 73.23879693616827, 38.23647082340259, 16.572248135413915]
